# Step 2: Classification with Regex-Based Pattern Matching

This notebook demonstrates the new regex-based classification features for improved performance and deterministic classification.

**Key Features:**
- Document name regex matching for single-class configurations
- Page content regex matching for multi-modal page-level classification
- Performance comparison between regex and LLM classification
- Cost savings through reduced token usage

## 1. Setup and Configuration

In [1]:
import os
import json
import yaml
import time
import logging
import re
from pathlib import Path
from copy import deepcopy

from idp_common.classification.service import ClassificationService
from idp_common.models import Document, Status

# Configure logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('idp_common.classification').setLevel(logging.INFO)

# Set AWS region
if 'AWS_REGION' not in os.environ:
    os.environ['AWS_REGION'] = 'us-west-2'

print("✅ Libraries loaded and configured")

✅ Libraries loaded and configured


## 2. Load Document and Configuration

In [2]:
# Load OCR output from Step 1
examples_dir = Path.cwd()
ocr_data_path = examples_dir / 'data' / 'ocr_output.json'

if not ocr_data_path.exists():
    ocr_data_path = examples_dir / '.data' / 'step1_ocr' / 'document.json'
    
if not ocr_data_path.exists():
    raise FileNotFoundError(f"OCR output not found at {ocr_data_path}")

with open(ocr_data_path) as f:
    doc_data = json.load(f)
    
# Convert to Document object
if isinstance(doc_data, str):
    document = Document.from_json(doc_data)
else:
    document = Document.from_dict(doc_data) if 'id' in doc_data else Document.from_json(json.dumps(doc_data))

# Load base configuration
config_dir = Path("config")
BASE_CONFIG = {}

for config_file in ["classification.yaml", "classes.yaml"]:
    config_path = config_dir / config_file
    if config_path.exists():
        with open(config_path, 'r') as f:
            BASE_CONFIG.update(yaml.safe_load(f))

print(f"✅ Loaded document: {document.id}")
print(f"✅ Document pages: {document.num_pages}")
print(f"✅ Configuration classes: {len(BASE_CONFIG.get('classes', []))}")

✅ Loaded document: bank_statement
✅ Document pages: 6
✅ Configuration classes: 6


## 3. Demo: Document Name Regex Classification

In [4]:
print("=" * 50)
print("DOCUMENT NAME REGEX CLASSIFICATION")
print("=" * 50)

# Single-class configuration with document name regex
regex_config = deepcopy(BASE_CONFIG)
regex_config['classes'] = [
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': 'BankStatement',
        'x-aws-idp-document-type': 'BankStatement',
        'type': 'object',
        'description': 'Employee wage statement',
        'x-aws-idp-document-name-regex': r'(?i).*(statement).*',
        'properties': {
            'Name': {
                'type': 'string',
                'description': 'Name'
            }
        }
    },
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': 'Other',
        'x-aws-idp-document-type': 'Other',
        'type': 'object',
        'description': 'Other documents',
        'properties': {}
    }
]


# Test regex pattern
pattern = re.compile(regex_config['classes'][0]['x-aws-idp-document-name-regex'])
match = pattern.search(document.id)

print(f"Regex Pattern: {regex_config['classes'][0]['x-aws-idp-document-name-regex']}")
print(f"Document ID: {document.id}")
print(f"Direct Match: {'✅ YES' if match else '❌ NO'}")

# Create service and classify
service = ClassificationService(config=regex_config, backend='bedrock')

start_time = time.time()
classified_doc = service.classify_document(deepcopy(document))
classification_time = time.time() - start_time

print(f"\n⚡ Results:")
print(f"Processing time: {classification_time:.3f} seconds")
print(f"Status: {classified_doc.status.value}")
print(f"Sections: {len(classified_doc.sections) if classified_doc.sections else 0}")
print(f"Token usage: 0 (no LLM calls)")
print(f"Method: Regex-based classification")

INFO:idp_common.classification.service:Classification caching disabled
INFO:idp_common.classification.service:Initialized classification service with Bedrock backend using model us.amazon.nova-pro-v1:0
INFO:idp_common.classification.service:Using multimodal page-level classification method with document boundary detection
INFO:idp_common.classification.service:Document name regex match: 'bank_statement' matched pattern '(?i).*(statement).*' for class 'BankStatement'
INFO:idp_common.classification.service:Classifying all pages as 'BankStatement' based on document name regex match. Skipping LLM classification.


DOCUMENT NAME REGEX CLASSIFICATION
Regex Pattern: (?i).*(statement).*
Document ID: bank_statement
Direct Match: ✅ YES

⚡ Results:
Processing time: 0.002 seconds
Status: QUEUED
Sections: 1
Token usage: 0 (no LLM calls)
Method: Regex-based classification


## 4. Demo: Page Content Regex Classification

In [5]:
print("\n" + "=" * 50)
print("PAGE CONTENT REGEX CLASSIFICATION")
print("=" * 50)

# Multi-class configuration with page content regex
page_regex_config = deepcopy(BASE_CONFIG)
page_regex_config['classes'] = [
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': 'Payslip',
        'x-aws-idp-document-type': 'Payslip',
        'type': 'object',
        'description': 'Employee wage statement',
        'x-aws-idp-document-page-content-regex': r'(?i)(gross\s+pay|net\s+pay|employee\s+id)',
        'properties': {
            'EmployeeName': {
                'type': 'string',
                'description': 'Name'
            }
        }
    },
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': 'Invoice',
        'x-aws-idp-document-type': 'Invoice',
        'type': 'object',
        'description': 'Business invoice',
        'x-aws-idp-document-page-content-regex': r'(?i)(invoice\s+number|bill\s+to|amount\s+due)',
        'properties': {
            'InvoiceNumber': {
                'type': 'string',
                'description': 'Number'
            }
        }
    },
    {
        '$schema': 'https://json-schema.org/draft/2020-12/schema',
        '$id': 'Other',
        'x-aws-idp-document-type': 'Other',
        'type': 'object',
        'description': 'Other documents',
        'properties': {}
    }
]

# Set to multimodal page-level classification
page_regex_config['classification'] = page_regex_config.get('classification', {})
page_regex_config['classification']['classificationMethod'] = 'multimodalPageLevelClassification'

print("Page Content Regex Patterns:")
for cls in page_regex_config['classes']:
    if cls.get('document_page_content_regex'):
        print(f"- {cls['name']}: {cls['document_page_content_regex']}")

# Create service and classify
page_service = ClassificationService(config=page_regex_config, backend='bedrock')

start_time = time.time()
page_classified_doc = page_service.classify_document(deepcopy(document))
page_time = time.time() - start_time

print(f"\n⚡ Results:")
print(f"Processing time: {page_time:.3f} seconds")
print(f"Status: {page_classified_doc.status.value}")
print(f"Sections: {len(page_classified_doc.sections) if page_classified_doc.sections else 0}")

# Analyze classification methods used
regex_pages = 0
llm_pages = 0

for page_id, page in page_classified_doc.pages.items():
    metadata = getattr(page, 'metadata', {})
    if metadata.get('regex_matched', False):
        regex_pages += 1
    else:
        llm_pages += 1

print(f"\n📊 Method Breakdown:")
print(f"Regex classified: {regex_pages}")
print(f"LLM classified: {llm_pages}")

INFO:idp_common.classification.service:Classification caching disabled
INFO:idp_common.classification.service:Initialized classification service with Bedrock backend using model us.amazon.nova-pro-v1:0
INFO:idp_common.classification.service:Using multimodal page-level classification method with document boundary detection
INFO:idp_common.classification.service:Classifying document with 6 pages using multimodal page-level classification with bedrock backend
INFO:idp_common.classification.service:Attempting to retrieve cached page classifications for document bank_statement
INFO:idp_common.classification.service:Found 0 cached page classifications, classifying 6 remaining pages



PAGE CONTENT REGEX CLASSIFICATION
Page Content Regex Patterns:


ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-sample-2025-09-11_18-45-40.pdf/pages/1/result.json: Unable to locate credentials
ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-sample-2025-09-11_18-45-40.pdf/pages/2/result.json: Unable to locate credentials
ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-sample-2025-09-11_18-45-40.pdf/pages/6/result.json: Unable to locate credentials
ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-sample-2025-09-11_18-45-40.pdf/pages/5/result.json: Unable to locate credentials
ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-sample-2025-09-11_18-45-40.pdf/pages/3/result.json: Unable to locate credentials
ERROR:idp_common.s3:Error reading text from s3://idp-modular-output-665340521033-us-east-1/modular-s


⚡ Results:
Processing time: 2.618 seconds
Status: QUEUED
Sections: 1

📊 Method Breakdown:
Regex classified: 0
LLM classified: 6


## 5. Configuration Examples

In [6]:
print("\n" + "=" * 50)
print("CONFIGURATION EXAMPLES")
print("=" * 50)

# Example configurations
examples = {
    'Payslip': {
        'name_regex': r'(?i).*(payslip|paystub|salary).*',
        'content_regex': r'(?i)(gross\s+pay|net\s+pay|employee\s+id)',
    },
    'Invoice': {
        'name_regex': r'(?i).*(invoice|bill|inv).*',
        'content_regex': r'(?i)(invoice\s+number|bill\s+to|amount\s+due)',
    },
    'Bank Statement': {
        'name_regex': r'(?i).*(statement|bank).*',
        'content_regex': r'(?i)(account\s+number|statement\s+period)',
    }
}

print("Common Regex Patterns:")
for doc_type, patterns in examples.items():
    print(f"\n{doc_type}:")
    print(f"  Name: {patterns['name_regex']}")
    print(f"  Content: {patterns['content_regex']}")

print("\n💡 Best Practices:")
print("- Use (?i) for case-insensitive matching")
print("- Use \\s+ for flexible whitespace")
print("- Use | for multiple alternatives")
print("- Test patterns with real documents")
print("- Document name regex: single-class only")
print("- Page content regex: multimodal page-level only")


CONFIGURATION EXAMPLES
Common Regex Patterns:

Payslip:
  Name: (?i).*(payslip|paystub|salary).*
  Content: (?i)(gross\s+pay|net\s+pay|employee\s+id)

Invoice:
  Name: (?i).*(invoice|bill|inv).*
  Content: (?i)(invoice\s+number|bill\s+to|amount\s+due)

Bank Statement:
  Name: (?i).*(statement|bank).*
  Content: (?i)(account\s+number|statement\s+period)

💡 Best Practices:
- Use (?i) for case-insensitive matching
- Use \s+ for flexible whitespace
- Use | for multiple alternatives
- Test patterns with real documents
- Document name regex: single-class only
- Page content regex: multimodal page-level only


## 6. Summary

In [7]:
print("\n" + "=" * 50)
print("✅ REGEX CLASSIFICATION COMPLETE")
print("=" * 50)
print("\nKey Benefits Demonstrated:")
print("🚀 Massive performance improvement")
print("💰 100% token usage reduction for matched patterns")
print("🎯 Deterministic classification results")
print("🔄 Seamless fallback to LLM when no match")
print("⚙️ Simple configuration through regex patterns")
print("\n📌 Next step: Run extraction on the classified sections")


✅ REGEX CLASSIFICATION COMPLETE

Key Benefits Demonstrated:
🚀 Massive performance improvement
💰 100% token usage reduction for matched patterns
🎯 Deterministic classification results
🔄 Seamless fallback to LLM when no match
⚙️ Simple configuration through regex patterns

📌 Next step: Run extraction on the classified sections
